In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install -U torch transformers tokenizers accelerate safetensors

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import re
import os
os.chdir('/content/drive/MyDrive/응용통계학과 공모전')

from matplotlib import rc
import matplotlib.font_manager as fm
# 디렉토리 및 파일 이름에 맞추어 변경
font_location ='./NanumGothic.ttf'
# 폰트 설정
rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

# 요약
1. 직무별 기술스택 그래프를 불러온다.
2. 확인하고 싶은 기술스택 그래프를 전역변수로 설정한다.
3. 사용자의 기술스택이 그래프 내에 존재하는지 확인한 뒤, 인접한 노드들을 gpt_chat_completion에 전달한 input으로 가져온다. 이때, 기술스택은 여러개가 전달될 수 있으므로 중복되는 기술스택은 제거한 뒤 input으로 가져온다.
4. 사용자의 기술스택이 그래프 내에 존재하지 않는다면, 그래프 내의 가중치가 가장 높은 상위 n개 노드를 가져와 input으로 가져온다.
5. role: system을 작성하며 단어를 바탕으로 문장을 출력하도록 프롬프트 엔지니어링을 진행한다.
6. 출력 결과를 확인한다.

In [ ]:
import numpy as np
import networkx as nx
import operator

from glob import glob

# #직무별 기술스택 그래프 불러오기
# for g in glob('data/text_network/*.gml'):
#     print(g.split('/')[-1].replace('_gpt.gml',''))
#     globals()[g.split('/')[-1].replace('_gpt.gml','')] = nx.read_gml(g)

#직무별 기술스택 그래프 불러오기
ML_graph = nx.read_gml("ML_graph_gpt.gml")
DE_graph = nx.read_gml("DE_graph_gpt.gml")
DSC_graph = nx.read_gml("DSC_graph_gpt.gml")
BD_graph = nx.read_gml("BD_graph_gpt.gml")
BI_graph = nx.read_gml("BI_graph_gpt.gml")
DBA_graph = nx.read_gml("DBA_graph_gpt.gml")

def get_neighbors(target, graph):
  try:
    connected_nodes = list(graph.neighbors(target))
  except:
    connected_nodes = []

  # 연결된 노드 중에서 가중치가 높은 상위 4개 노드 출력
  if connected_nodes:
      weights = [(node, graph[target][node]['weight']) for node in connected_nodes]
      sorted_weights = sorted(weights, key=operator.itemgetter(1), reverse=True)
      print(sorted_weights[:4])
      return sorted_weights[:4]

  else:
      print(f"No nodes connected to '{target}'.")
      return -1

#만약 사용자 input이 graph내에 하나도 없을 경우 weight가 높은 순으로 node를 반환
def get_sorted_nodes(graph):

  edge_weights = nx.get_edge_attributes(graph, 'weight')

  # 가중치가 높은 순으로 노드 정렬
  sorted_nodes = sorted(edge_weights, key=edge_weights.get, reverse=True)

  return sorted_nodes[:10]

# Ko-Alpaca 모델을 이용 (채택 x)

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM

MODEL = 'beomi/KoAlpaca-Polyglot-5.8B'

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model.eval()

pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=MODEL,
    device='cuda'
)

def ask(x, context='', is_input_full=False):
    ans = pipe(
        f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:",
        do_sample=True,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    print(ans[0]['generated_text'])

ask("딥러닝이 뭐야?")
# 딥러닝은 인공신경망을 통해 입력과 출력 사이의 복잡한 관계를 학습하는 머신러닝의 한 분야입니다. 이 기술은 컴퓨터가 인간의 학습 능력과 유사한 방식으로 패턴을 학습하도록 하며, 인간의 개입 없이도 데이터를 처리할 수 있는 기술입니다. 최근에는 딥러닝을 활용한 인공지능 애플리케이션이 많이 개발되고 있습니다. 예를 들어, 의료 진단 애플리케이션에서는 딥러닝 기술을 활용하여 환자의 특징을 파악하고, 이를 통해 빠르고 정확한 진단을 내리는 데 사용됩니다. 또한, 금융 분야에서는 딥러닝 기술을 활용하여 주가 예측 모형을 학습하는 데 사용되기도 합니다.

# OPENAI gpt-3.5 turbo모델 이용

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import openai

# OpenAI API 키 설정 (자신의 API 키로 대체해야 함)
api_key = ''
openai.api_key = api_key

# GPT-3 API 호출 함수
def generate_text(skill):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages= [
            {"role": "system", "content": "당신은 입력된 단어를 이용해서, 사용자에게 관련된 역량을 길러보라고 알려주는 machine입니다."},
            {"role": "system", "content": "단어에 대한 설명은 생략하고, 짧은 문장으로만 답변을 제시하세요."},
            {"role": "system", "content": "입력받은 단어가 최대한 들어가도록 문장을 구성하세요."},
            {"role": "system", "content": "만약 React와 같이 기술스택과 관련한 단어를 입력받았다면, 당신의 답변은 'React 프레임워크 기술을 쌓아보세요.'와 같은 문장이어야 합니다."},
            {"role": "system", "content": "학위와 관련된 단어를 입력받았다면, 당신의 답변은 '전문적인 지식이 요구되는 분야이니, 석사 이상의 학위 취득을 추천드립니다.'와 같은 문장이어야 합니다. "},
            {"role": "system", "content": "경험 혹은 역량과 관련된 단어라면, 관련된 경험을 쌓아보라는 문장을 제시해야 합니다."},
            {"role": "system", "content": "한글로 답변하세요."},
            {"role": "user", "content" : skill}
        ],
        temperature=0.5
    )
    return response.choices[0].message.content

## user에게 input으로 기술스택 받았다고 하자

In [ ]:
skill_set = input("사용 가능하신 기술스택을 나열해 주세요. ex) python, sql ... ")

사용 가능하신 기술스택을 나열해 주세요. ex) python, sql ... python, sql


In [ ]:
#전역변수
GRAPH = ML_graph

#주변 노드가 담길 list
neighbor_list = []
extracted_nodes = []

#input의 주변 노드를 구함
for skill in skill_set.split(','):
  tmp = get_neighbors(skill.strip(), GRAPH)
  if tmp != -1:
    neighbor_list.append(tmp)

#주변 노드를 담는 리스트가 비어있지 않으면
if neighbor_list:
  # gpt 모델에 전달할 기술스택 리스트를 뽑는 과정
  for i in range(len(neighbor_list)):
    for j in range(4):
      extracted_nodes.append(neighbor_list[i][j][0])


#사용자의 스킬이 그래프 내에 존재하지 않아서 주변 노드 리스트가 비어있는 경우
else:
  rank = get_sorted_nodes(GRAPH)

  for i in range(len(rank)):
    extracted_nodes.append(rank[i][0])
    extracted_nodes.append(rank[i][1])

# extracted_nodes에서 사용자에게 인풋으로 받은 기술스택은 추천하지 않도록 제거
skill_set_list = [value.strip() for value in skill_set.split(',')]
extracted_nodes = [value for value in extracted_nodes if value not in skill_set_list]
print(skill_set_list)
print(extracted_nodes)

gpt_input = ",".join(list(set(extracted_nodes)))
print(gpt_input)
#gpt 답변
generated_sentence = generate_text(gpt_input)

# 문장을 점(.)으로 나누기
sentences = generated_sentence.split('.')

# 공백을 제거하고 문장을 다시 조립
output = "\n".join(sentence.strip() + '.' for sentence in sentences if sentence)

print(output)

[('pytorch', 61), ('aws', 48), ('tensorflow', 40), ('git', 38)]
[('aws', 17), ('python', 14), ('git', 9), ('linux', 9)]
['python', 'sql']
['pytorch', 'aws', 'tensorflow', 'git', 'aws', 'git', 'linux']
git,linux,pytorch,aws,tensorflow
Git을 이용하여 협업 및 버전 관리 능력을 향상시켜보세요.
Linux 환경에서 개발을 해보면 다양한 도구와 명령어에 익숙해질 수 있습니다.
PyTorch를 사용해 딥러닝 역량을 키워보세요.
AWS를 활용하여 클라우드 컴퓨팅 및 인프라 관리 능력을 향상시켜보세요.
TensorFlow를 사용하여 머신러닝 역량을 키워보세요.


In [ ]:
output = generate_text('분산환경, 의사결정, 최적화')
print(output)

분산환경에 대한 역량을 길러보세요. 의사결정과 최적화 역량도 함께 강화해보면 좋을 것 같습니다.
